In [28]:
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Configurar la fecha de valuación
settle = datetime.strptime('07-may-2024', '%d-%b-%Y')

# Plazos y tasas de interés
plazos = np.array([0.5, 1, 2, 3, 4, 5, 7, 10])
tasas = np.array([5.3825, 5.2320, 5.0164, 4.8588, 4.7751, 4.6914, 4.6752, 4.6500]) / 100

# Calcular fechas para la curva de tasas
fechas_tasas = [settle + timedelta(days=int(360 * p)) for p in plazos]

# Construir la curva de tasas (fechas y tasas)
curva_tasas = list(zip(fechas_tasas, tasas))

# Probabilidades de default (constantes para todas las fechas)
probas_default = 0.062070
probas = [(fecha, probas_default) for fecha in fechas_tasas]

# Parámetros del CDS
maturity = datetime.strptime('07-may-2026', '%d-%b-%Y')
recovery_rate = 0.47

# Generar fechas de pago del CDS (trimestrales)
pmtdates = []
current_date = settle + relativedelta(months=3)
while current_date <= maturity:
    pmtdates.append(current_date)
    current_date += relativedelta(months=3)
if not pmtdates or pmtdates[-1] != maturity:
    pmtdates.append(maturity)

# Función para interpolar tasas
def interpolar_tasa(t):
    if t <= plazos[0]:
        return tasas[0]
    elif t >= plazos[-1]:
        return tasas[-1]
    else:
        return np.interp(t, plazos, tasas)

# Calcular tiempos desde 'settle' en años
tiempos = [(date - settle).days / 365.25 for date in pmtdates]

# Interpolar tasas y calcular factores de descuento
tasas_interpoladas = [interpolar_tasa(t) for t in tiempos]
discount_factors = np.exp(-np.array(tasas_interpoladas) * np.array(tiempos))

# Calcular probabilidades de supervivencia (1 - PD)
survival_probs = [1 - probas_default] * len(pmtdates)

# Calcular fracciones de tiempo entre pagos (ACT/360)
deltas = []
prev_date = settle
for date in pmtdates:
    delta_days = (date - prev_date).days
    deltas.append(delta_days / 360.0)
    prev_date = date

# PV de la pata premium
premium_leg = np.sum(discount_factors * survival_probs * deltas)

# PV de la pata de protección (defaults entre fechas)
protection_leg = 0.0
prev_q = 0.0
for i in range(len(pmtdates)):
    q = probas_default  # PD acumulada (incorrecto pero según código original)
    protection_leg += (1 - recovery_rate) * (q - prev_q) * discount_factors[i]
    prev_q = q

# Calcular el spread del CDS
spread = protection_leg / premium_leg

print(f"Spread del CDS: {spread * 10000:.2f} puntos básicos")

Spread del CDS: 180.73 puntos básicos
